<a href="https://colab.research.google.com/github/qortmdgh4141/Digital-Therapeutics-Platform-Development/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Package Imports and Shared Function Definitions

In [ ]:
from keras import initializers
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Conv3D, MaxPooling3D, MaxPooling2D, Conv2D, BatchNormalization, Conv3DTranspose, Permute, Activation, LeakyReLU
from keras import regularizers

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow import keras

from google.colab import files

import os
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def train_test_dir_path(dir_path):
    split_path = dir_path.split('/')
    feature = split_path[-2]  # 맨 오른쪽에서 두번째에 있는 "distance"
    feature_train_test = split_path[-1]  # 맨 오른쪽 첫번째에 있는 문자열 "distance(1&5-3)"

    extracted = feature_train_test[feature_train_test.find("(")+1 : feature_train_test.find(")")]  # "()" 안의 문자열 "1&5-3"
    split_extracted = extracted.split("-")
    train_part = split_extracted[0]  # "1&5"
    test_part = split_extracted[1]  # "3"

    feature_train = 'train_' + feature + "(" + train_part + ")"
    feature_test = 'test_' + feature + "(" + test_part + ")"

    train_dir_path = os.path.join(dir_path, feature_train)
    test_dir_path = os.path.join(dir_path, feature_test)

    return train_dir_path, test_dir_path, feature_train_test
    
def plot_loss_and_accuracy(train_loss, val_loss, train_acc, val_acc, model_name):
    epochs = range(1, len(train_loss) + 1)
    
    plt.figure(figsize=(12, 6))
    
    # Loss 그래프
    plt.subplot(2, 2, 1)
    plt.plot(epochs, train_loss, 'b', label='Training Loss')
    plt.plot(epochs, val_loss, 'r', label='Validation Loss')
    plt.title(f'{model_name} Model - Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Accuracy 그래프
    plt.subplot(2, 2, 2)
    plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
    plt.title(f'{model_name} Model - Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    
    plt.show()

## Defining the CNN Classifier Class

In [ ]:
class cnn_clf:
    def __init__(self, train_dir_path, test_dir_path):
        self.train_dir_path = train_dir_path
        self.test_dir_path = test_dir_path

    def main(self, augmentation, total_angle_num, angle_interval, element):
        x_train, y_train = self.load_dataset(dir_path=self.train_dir_path, mode="TRAIN", shuffle=True)
        x_test, y_test = self.load_dataset(dir_path=self.test_dir_path, mode="TEST", shuffle=False)

        x_train = self.feature_scaling(x_train, norm=False, std=True, with_mean=True)
        x_test = self.feature_scaling(x_test, norm=False, std=True, with_mean=True)

        if augmentation == True:
            x_train, y_train, augment_num_angles = self.augment_data(x_train, y_train, shuffle=True)
            x_test, y_test, augment_num_angles = self.augment_data(x_test, y_test, shuffle=False)
        else:
            augment_num_angles = 1

        if angle_interval != 10:
            x_train = self.reshape_by_angle_interval(x_train, total_angle_num=total_angle_num, angle_interval=angle_interval)
            x_test = self.reshape_by_angle_interval(x_test, total_angle_num=total_angle_num, angle_interval=angle_interval)

        cnn_result, train_val_score, cnn_model  = self.train_test_cnn(x_train, y_train, x_test, y_test, element, augment_num_angles)

        return cnn_result, train_val_score, cnn_model 

    def load_dataset(self, dir_path, mode, shuffle):
        x_train_list = []
        y_train = np.array([])

        if shuffle == True:
            random.seed(42)
            sample_size = len(os.listdir(dir_path))
            population = list(range(sample_size))
            samples_num = random.sample(population, sample_size)
        else :
            sample_size = len(os.listdir(dir_path))
            samples_num = list(range(sample_size))

        for num in samples_num:
            loaded = np.load(f"{dir_path}/{mode}_({num}).npz")
            data = loaded["data"]
            label = loaded["label"]

            x_train_list.append(data)
            y_train = np.concatenate([y_train, label])

        x_train = np.stack(x_train_list, axis=0).astype(np.float32)
        y_train = y_train.astype(np.float32)

        return x_train, y_train

    def feature_scaling(self, data, norm, std, with_mean):
        if std == True:
            if with_mean==True:
                mean = np.mean(data, axis=tuple(range(data.ndim)), keepdims=True)
                std = np.std(data, axis=tuple(range(data.ndim)), keepdims=True)
                data = (data - mean) / std
            else:
                std = np.std(data, axis=tuple(range(data.ndim)), keepdims=True)
                data = data / std
        elif norm == True:
            min = data.min()
            max = data.max()
            data = (data - min) / (max - min)

        return data

    def augment_data(self, x_data, y_data, shuffle):
        num_samples = x_data.shape[0]  # 배치 사이즈
        num_angles = x_data.shape[1]  # 각도의 개수

        # 어규멘테이션된 데이터를 저장할 변수 초기화
        augmented_x_data = np.zeros((num_samples * num_angles, *x_data.shape[1:]), dtype=x_data.dtype)
        augmented_y_data = np.zeros((num_samples * num_angles), dtype=x_data.dtype)

        for i in range(num_samples):
            for j in range(num_angles):
                # 어규멘테이션 데이터
                augmented_x_data[i * num_angles + j] = np.roll(x_data[i], -j, axis=0)
                augmented_y_data[i * num_angles + j] = y_data[i]

        # 라벨 매핑 검증
        for i in range(num_samples):
            start_index = i * num_angles
            end_index = (i + 1) * num_angles
            labels = np.unique(augmented_y_data[start_index:end_index])
            if labels[0] != y_data[i]:
                print("Label mapping error for x_train[{}], y_train[{}].".format(i, i))

        # augmented_x_train의 배치사이즈를 기준으로 데이터 순서를 셔플
        if shuffle == True:
            shuffle_indices = np.random.permutation(augmented_x_data.shape[0])
            augmented_x_data = augmented_x_data[shuffle_indices]
            augmented_y_data = augmented_y_data[shuffle_indices]

        print("\t - Original to Augmented 'x_data / y_data' shape:", x_data.shape, "/", y_data.shape, "->", augmented_x_data.shape, "/", augmented_y_data.shape)

        return augmented_x_data, augmented_y_data, num_angles

    def reshape_by_angle_interval(self, data, total_angle_num, angle_interval):
        angle_num = (total_angle_num * 10) // angle_interval
        angle_indices = np.linspace(0, total_angle_num - 1, angle_num, dtype=int) # 선택할 각도 인덱스 계산
        new_data = np.zeros((data.shape[0], angle_num, *data.shape[2:]), dtype=data.dtype) # 새로운 데이터셋 생성

        # 각도 인덱스를 이용하여 데이터 복사
        for i, angle_idx in enumerate(angle_indices):
            new_data[:, i, :] = data[:, angle_idx, :]

        print(f"\t - X_Data reshaped by angle interval (angle interval - {angle_interval} degree) : ", new_data.shape)

        return new_data
    
    def train_test_cnn(self, x_train, y_train, x_test, y_test, element, augment_num_angles):
        input_data_shape = x_train.shape[1:]
        cnn_model = self.create_cnn_model(input_data_shape)

        print("CNN Model - RUNNING".center(80, "="))

        # 학습 
        lr_scheduler = LearningRateScheduler(self.exponential_decay)
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, restore_best_weights=True)        
        cnn_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])
        train_val_score = cnn_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, callbacks=[lr_scheduler, early_stopping]) 
        
        # 학습된 모형 테스트 
        if element == None: 
            test_score = cnn_model.evaluate(x_test, y_test)
            test_accuracy = round(test_score[1]*100)
            
            print(f"\nTest Accuracy: {test_accuracy}%")

            return test_accuracy

        else:
            count = 0
            _results = np.zeros(6)
            x_test_all, y_test_all= self.split_y_test(x_test , y_test, element=element, augment_num_angles=augment_num_angles)

            for num_1 in range(len(y_test_all)):
                for num_2 in range(len(y_test_all[0])):
                    test_score = cnn_model.evaluate(x_test_all[num_1][num_2], y_test_all[num_1][num_2])
                    test_accuracy = round(test_score[1]*100)
                    _results[count] = test_accuracy
                    count+=1

            mean_value = np.mean(_results[0:3])
            cnn_result = np.round(np.insert(_results, 0, mean_value))

            print(f"Test Accuracy : {cnn_result[0]}%   ==>   ({', '.join([f'{acc}%' for acc in cnn_result[1:4]])}  /  {', '.join([f'{acc}%' for acc in cnn_result[4:7]])})")

            return cnn_result, train_val_score, cnn_model

    def split_y_test(self, x_test, y_test, element, augment_num_angles):
        if element != None:
            indices_1 = [0, 1, 2, 9, 10, 11]; indices_2 = [3, 4, 5, 12, 13, 14]; indices_3 = [6, 7, 8, 15, 16, 17]
            ind_1 = [0, 3, 6, 9, 12, 15]; ind_2 = [1, 4, 7, 10, 13, 16]; ind_3 = [2, 5, 8, 11, 14, 17]
            aug_indices_list = [np.array([], dtype=int) for _ in range(3)]
            aug_ind_list = [np.array([], dtype=int) for _ in range(3)]

            for num, indices in enumerate([indices_1, indices_2, indices_3]):
                for i in indices:
                    aug_indices_list[num] = np.concatenate((aug_indices_list[num], np.arange(i * augment_num_angles, i * augment_num_angles + augment_num_angles)))
            for num, ind in enumerate([ind_1, ind_2, ind_3]):
                for i in ind:
                    aug_ind_list[num] = np.concatenate((aug_ind_list[num], np.arange(i * augment_num_angles, i * augment_num_angles + augment_num_angles)))

            x_test_1 = [x_test[aug_indices_list[0]], x_test[aug_indices_list[1]], x_test[aug_indices_list[2]]]
            y_test_1 = [y_test[aug_indices_list[0]], y_test[aug_indices_list[1]], y_test[aug_indices_list[2]]]
            x_test_2 = [x_test[aug_ind_list[0]], x_test[aug_ind_list[1]], x_test[aug_ind_list[2]]]
            y_test_2 = [y_test[aug_ind_list[0]], y_test[aug_ind_list[1]], y_test[aug_ind_list[2]]]

            x_test_all = [x_test_1, x_test_2]
            y_test_all = [y_test_1, y_test_2]

            return x_test_all, y_test_all

    def exponential_decay(self, epoch):
        initial_lr = 0.1; decay_rate = 0.1
        lr = initial_lr * np.exp(-decay_rate * epoch)
        return lr

    def step_decay(self, epoch):
        initial_lr = 0.1; drop_rate = 0.5; drop_epochs = 10
        lr = initial_lr * (drop_rate ** (epoch // drop_epochs))
        return lr

    def create_cnn_model(self, input_data_shape):
        cnn_model = Sequential()

        cnn_model.add(Conv3D(32, kernel_size=(3, 3, 3), strides=(1, 1, 2), padding="same", input_shape=input_data_shape, kernel_initializer=initializers.HeNormal(), data_format="channels_first"))
        cnn_model.add(BatchNormalization())
        cnn_model.add(LeakyReLU(alpha=0.01))
        cnn_model.add(Dropout(0.7))
        cnn_model.add(MaxPooling3D(pool_size=(1, 1, 5), data_format="channels_first"))

        cnn_model.add(Flatten())

        cnn_model.add(Dense(32, kernel_initializer=initializers.HeNormal()))
        cnn_model.add(BatchNormalization())
        cnn_model.add(LeakyReLU(alpha=0.01))
        cnn_model.add(Dropout(0.7))

        cnn_model.add(Dense(1, activation='sigmoid'))

        return cnn_model

## Training the CNN Model - Setup

In [ ]:
all_dir_path = []

dir_paths = [os.path.join("/content/drive/MyDrive/Colab Notebooks/DTX/merged_sample3/all/angle360_step10", feature) for feature in ["distance", "height", "type"]]

# 기본 36개의 각도를 다 사용하는 angle_interval = 10
angle_interval = 10

split_accuracy = True
augmentation = True
total_angle_num = 36

In [ ]:
for dir_path in dir_paths :
    subdir_name = next(os.walk(dir_path))[1]
    for name in subdir_name:
        subdir_path = os.path.join(dir_path, name)
        all_dir_path.append(subdir_path)

# 컬럼명만 있는 데이터프레임 생성
if split_accuracy==True:
    result_df = pd.DataFrame(columns=['Train/Test Set Composition', 'CNN Accuracy', "Accuracy_1-1", "Accuracy_1-2", "Accuracy_1-3", "Accuracy_2-1", "Accuracy_2-2", "Accuracy_2-3"])
else:
    result_df = pd.DataFrame(columns=['Train/Test Set Composition', 'CNN Accuracy'])

# 빈 행 추가
empty_row = pd.DataFrame([{}], columns=result_df.columns)
result_df = pd.concat([result_df, empty_row], ignore_index=True)

## Training the CNN Model

In [ ]:
iter_num = 7  # 호호
train_val_score_list = [None]*iter_num
model_list = [None]*iter_num

for num, dir_path in enumerate(all_dir_path):
    if (num+1) == iter_num: pass # 하나씩 저장
    else : continue

    (train_path, test_path, train_test_set_composition) = train_test_dir_path(dir_path)

    print(f"Start ({num+1} / {len(all_dir_path)})     :       {train_test_set_composition}    ({dir_path})")
    start_time = time.time()
    cnn = cnn_clf(train_path, test_path)

    if split_accuracy == True:
        element = train_test_set_composition.split('(')[0]
        cnn_result, train_val_score_list[num], model_list[num] = cnn.main(augmentation=augmentation, element=element, total_angle_num=total_angle_num, angle_interval=angle_interval)
    else :
        element = None
        cnnr_result, train_val_score_list[num], model_list[num] = cnn.main(augmentation=augmentation, element=element, total_angle_num=total_angle_num, angle_interval=angle_interval)

    # 새로운 값 추가
    if split_accuracy == True:
        new_values = [train_test_set_composition, cnn_result[0], cnn_result[1], cnn_result[2], cnn_result[3], cnn_result[4], cnn_result[5], cnn_result[6]]
        result_df.loc[result_df.index[-1] + 1] = new_values
    else:
        new_values = [train_test_set_composition, cnn_result]
        result_df.loc[result_df.index[-1] + 1] = new_values

    # 빈 행 추가
    if (num - 2) % 3 == 0:
        empty_row = pd.DataFrame([{}], columns=result_df.columns)
        result_df = pd.concat([result_df, empty_row], ignore_index=True)

    elapsed_time = time.time() - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    print(f"Time elapsed: {minutes} minutes {seconds} seconds\n")
    print("\n")
    #break

In [ ]:
# CNN 모델 결과 그래프 출력 : Test Accuracy : 72.0%   ==>   (100.0%, 67.0%, 50.0%  /  83.0%, 67.0%, 67.0%)
plot_loss_and_accuracy(train_val_score_list[iter_num-1].history['loss'], train_val_score_list[iter_num-1].history['val_loss'],
                       train_val_score_list[iter_num-1].history['accuracy'], train_val_score_list[iter_num-1].history['val_accuracy'], 'CNN')

## Saving the Pretrained Model, Loss, Accuracy

In [ ]:
# save_model_path = '/content/drive/MyDrive/Colab Notebooks/DTX/pretrained_model_weights/all/angle360_step10/distance/distance(3&5-1)' #호호
# save_model_path = '/content/drive/MyDrive/Colab Notebooks/DTX/pretrained_model_weights/all/angle360_step10/height/height(2&4-0)' #호호
save_model_path = '/content/drive/MyDrive/Colab Notebooks/DTX/pretrained_model_weights/all/angle360_step10/type/type(1.5&2-4)' #호호

# 테스트 데이터 정확도 저장
result_df.to_excel('sample_result_all.xlsx', index=False)
files.download('sample_result_all.xlsx')

# 학습 완료된 모델 가중치 저장
for i, model in enumerate(model_list):
    if i==(iter_num-1): # 하나씩 저장
        model.save(f'{save_model_path}/model{i}.h5')

# 학습 완료된 모델의 학습과정(loss & accuracy) 저장
for i, train_val_score in enumerate(train_val_score_list):
    if i==(iter_num-1): # 하나씩 저장
        loss_values = train_val_score.history['loss']
        accuracy_values = train_val_score.history['accuracy']
        val_loss_values = train_val_score.history['val_loss']
        val_accuracy_values = train_val_score.history['val_accuracy']

        np.savetxt(f'{save_model_path}/train_loss{i}.txt', loss_values)
        np.savetxt(f'{save_model_path}/train_accuracy{i}.txt', accuracy_values)
        np.savetxt(f'{save_model_path}/val_loss{i}.txt', val_loss_values)
        np.savetxt(f'{save_model_path}/val_accuracy{i}.txt', val_accuracy_values)

In [ ]:
# save_model_path = '/content/drive/MyDrive/Colab Notebooks/DTX/pretrained_model_weights/ALL/angle360_step10/distance/distance(1&3-5)'

# 손실(loss) 값을 파일로부터 불러오기

train_loss_file_path = f'{save_model_path}/train_loss{(iter_num-1)}.txt'
train_loss_v = np.loadtxt(train_loss_file_path)
val_loss_file_path = f'{save_model_path}/val_loss{(iter_num-1)}.txt'
val_loss_v = np.loadtxt(val_loss_file_path)

# 정확도(accuracy) 값을 파일로부터 불러오기
train_accuracy_file_path = f'{save_model_path}/train_accuracy{(iter_num-1)}.txt'
train_acc_v = np.loadtxt(train_accuracy_file_path)
val_accuracy_file_path = f'{save_model_path}/val_accuracy{(iter_num-1)}.txt'
val_acc_v = np.loadtxt(val_accuracy_file_path)

# CNN 모델 결과 그래프 출력
plot_loss_and_accuracy(train_loss_v, val_loss_v,
                       train_acc_v, val_acc_v, 'CNN')

## Test : Defining the Pretrainded CNN Classifier Class 

In [ ]:
class pretrained_cnn_clf:
    def __init__(self, train_dir_path, test_dir_path, pretrained_cnn_model):
        self.train_dir_path = train_dir_path
        self.test_dir_path = test_dir_path
        self.cnn_model = pretrained_cnn_model

    def main(self, augmentation, total_angle_num, angle_interval, element):
        x_train, y_train = self.load_dataset(dir_path=self.train_dir_path, mode="TRAIN", shuffle=True)
        x_test, y_test = self.load_dataset(dir_path=self.test_dir_path, mode="TEST", shuffle=False)

        x_train = self.feature_scaling(x_train, norm=False, std=True, with_mean=True)
        x_test = self.feature_scaling(x_test, norm=False, std=True, with_mean=True)

        if augmentation == True:
            x_train, y_train, augment_num_angles = self.augment_data(x_train, y_train, shuffle=True)
            x_test, y_test, augment_num_angles = self.augment_data(x_test, y_test, shuffle=False)
        else:
            augment_num_angles = 1

        if angle_interval != 10:
            x_train = self.reshape_by_angle_interval(x_train, total_angle_num=total_angle_num, angle_interval=angle_interval)
            x_test = self.reshape_by_angle_interval(x_test, total_angle_num=total_angle_num, angle_interval=angle_interval)

        cnn_result = self.train_test_cnn(x_train, y_train, x_test, y_test, element, augment_num_angles)

        return cnn_result

    def load_dataset(self, dir_path, mode, shuffle):
        x_train_list = []
        y_train = np.array([])

        if shuffle == True:
            random.seed(42)
            sample_size = len(os.listdir(dir_path))
            population = list(range(sample_size))
            samples_num = random.sample(population, sample_size)
        else :
            sample_size = len(os.listdir(dir_path))
            samples_num = list(range(sample_size))

        for num in samples_num:
            loaded = np.load(f"{dir_path}/{mode}_({num}).npz")
            data = loaded["data"]
            label = loaded["label"]

            x_train_list.append(data)
            y_train = np.concatenate([y_train, label])

        x_train = np.stack(x_train_list, axis=0).astype(np.float32)
        y_train = y_train.astype(np.float32)

        return x_train, y_train

    def feature_scaling(self, data, norm, std, with_mean):
        if std == True:
            if with_mean==True:
                mean = np.mean(data, axis=tuple(range(data.ndim)), keepdims=True)
                std = np.std(data, axis=tuple(range(data.ndim)), keepdims=True)
                data = (data - mean) / std
            else:
                std = np.std(data, axis=tuple(range(data.ndim)), keepdims=True)
                data = data / std
        elif norm == True:
            min = data.min()
            max = data.max()
            data = (data - min) / (max - min)

        return data

    def augment_data(self, x_data, y_data, shuffle):
        num_samples = x_data.shape[0]  # 배치 사이즈
        num_angles = x_data.shape[1]  # 각도의 개수

        # 어규멘테이션된 데이터를 저장할 변수 초기화
        augmented_x_data = np.zeros((num_samples * num_angles, *x_data.shape[1:]), dtype=x_data.dtype)
        augmented_y_data = np.zeros((num_samples * num_angles), dtype=x_data.dtype)

        for i in range(num_samples):
            for j in range(num_angles):
                # 어규멘테이션 데이터
                augmented_x_data[i * num_angles + j] = np.roll(x_data[i], -j, axis=0)
                augmented_y_data[i * num_angles + j] = y_data[i]

        # 라벨 매핑 검증
        for i in range(num_samples):
            start_index = i * num_angles
            end_index = (i + 1) * num_angles
            labels = np.unique(augmented_y_data[start_index:end_index])
            if labels[0] != y_data[i]:
                print("Label mapping error for x_train[{}], y_train[{}].".format(i, i))

        # augmented_x_train의 배치사이즈를 기준으로 데이터 순서를 셔플
        if shuffle == True:
            shuffle_indices = np.random.permutation(augmented_x_data.shape[0])
            augmented_x_data = augmented_x_data[shuffle_indices]
            augmented_y_data = augmented_y_data[shuffle_indices]

        print("\t - Original to Augmented 'x_data / y_data' shape:", x_data.shape, "/", y_data.shape, "->", augmented_x_data.shape, "/", augmented_y_data.shape)

        return augmented_x_data, augmented_y_data, num_angles

    def reshape_by_angle_interval(self, data, total_angle_num, angle_interval):
        angle_num = (total_angle_num * 10) // angle_interval
        angle_indices = np.linspace(0, total_angle_num - 1, angle_num, dtype=int) # 선택할 각도 인덱스 계산
        new_data = np.zeros((data.shape[0], angle_num, *data.shape[2:]), dtype=data.dtype) # 새로운 데이터셋 생성

        # 각도 인덱스를 이용하여 데이터 복사
        for i, angle_idx in enumerate(angle_indices):
            new_data[:, i, :] = data[:, angle_idx, :]

        print(f"\t - X_Data reshaped by angle interval (angle interval - {angle_interval} degree) : ", new_data.shape)

        return new_data
    
    def train_test_cnn(self, x_train, y_train, x_test, y_test, element, augment_num_angles):

        print("Test Pretrained CNN Model - RUNNING".center(80, "="))
        
        # 학습된 모형 테스트 
        if element == None: 
            test_score = self.cnn_model.evaluate(x_test, y_test)
            test_accuracy = round(test_score[1]*100)
            
            print(f"\nTest Accuracy: {test_accuracy}%")

            return test_accuracy

        else:
            count = 0
            _results = np.zeros(6)
            x_test_all, y_test_all= self.split_y_test(x_test , y_test, element=element, augment_num_angles=augment_num_angles)

            for num_1 in range(len(y_test_all)):
                for num_2 in range(len(y_test_all[0])):
                    test_score = self.cnn_model.evaluate(x_test_all[num_1][num_2], y_test_all[num_1][num_2])
                    test_accuracy = round(test_score[1]*100)
                    _results[count] = test_accuracy
                    count+=1

            mean_value = np.mean(_results[0:3])
            cnn_result = np.round(np.insert(_results, 0, mean_value))

            print(f"Test Accuracy : {cnn_result[0]}%   ==>   ({', '.join([f'{acc}%' for acc in cnn_result[1:4]])}  /  {', '.join([f'{acc}%' for acc in cnn_result[4:7]])})")

            return cnn_result

    def split_y_test(self, x_test, y_test, element, augment_num_angles):
        if element != None:
            indices_1 = [0, 1, 2, 9, 10, 11]; indices_2 = [3, 4, 5, 12, 13, 14]; indices_3 = [6, 7, 8, 15, 16, 17]
            ind_1 = [0, 3, 6, 9, 12, 15]; ind_2 = [1, 4, 7, 10, 13, 16]; ind_3 = [2, 5, 8, 11, 14, 17]
            aug_indices_list = [np.array([], dtype=int) for _ in range(3)]
            aug_ind_list = [np.array([], dtype=int) for _ in range(3)]

            for num, indices in enumerate([indices_1, indices_2, indices_3]):
                for i in indices:
                    aug_indices_list[num] = np.concatenate((aug_indices_list[num], np.arange(i * augment_num_angles, i * augment_num_angles + augment_num_angles)))
            for num, ind in enumerate([ind_1, ind_2, ind_3]):
                for i in ind:
                    aug_ind_list[num] = np.concatenate((aug_ind_list[num], np.arange(i * augment_num_angles, i * augment_num_angles + augment_num_angles)))

            x_test_1 = [x_test[aug_indices_list[0]], x_test[aug_indices_list[1]], x_test[aug_indices_list[2]]]
            y_test_1 = [y_test[aug_indices_list[0]], y_test[aug_indices_list[1]], y_test[aug_indices_list[2]]]
            x_test_2 = [x_test[aug_ind_list[0]], x_test[aug_ind_list[1]], x_test[aug_ind_list[2]]]
            y_test_2 = [y_test[aug_ind_list[0]], y_test[aug_ind_list[1]], y_test[aug_ind_list[2]]]

            x_test_all = [x_test_1, x_test_2]
            y_test_all = [y_test_1, y_test_2]

            return x_test_all, y_test_all

## Test : Pretrainded CNN Model

In [ ]:
# 기본 36개의 각도를 다 사용하는 angle_interval = 10
pre_angle_interval = 10

pre_split_accuracy = True
pre_augmentation = True
pre_total_angle_num = 36

# 저장된 모델 불러오기

loaded_model = keras.models.load_model(f'{save_model_path}/model{iter_num-1}.h5')
pretrained_cnn = pretrained_cnn_clf(pre_train_path, pre_test_path, pretrained_cnn_model=loaded_model)

# pre_dir_path = '/content/drive/MyDrive/Colab Notebooks/DTX/merged_sample3/all/angle360_step10/distance/distance(3&5-1)' #호호
# pre_dir_path = '/content/drive/MyDrive/Colab Notebooks/DTX/merged_sample3/all/angle360_step10/height/height(0&4-2)' #호호
pre_dir_path = '/content/drive/MyDrive/Colab Notebooks/DTX/merged_sample3/all/angle360_step10/type/type(1.5&2-4)' #호호

(pre_train_path, pre_test_path, pre_train_test_set_composition) = train_test_dir_path(pre_dir_path)

if split_accuracy == True:
    pre_element = pre_train_test_set_composition.split('(')[0]
    pretrained_cnn.main(augmentation=pre_augmentation, element=pre_element, total_angle_num=pre_total_angle_num, angle_interval=pre_angle_interval)
else :
    pre_element = None
    pretrained_cnn.main(augmentation=pre_augmentation, element=pre_element, total_angle_num=pre_total_angle_num, angle_interval=pre_angle_interval)